In [1]:
from lsst.daf.butler import Butler
from lsst.obs.base import DefineVisitsTask, Instrument


In [2]:
# I think we're talking about ComCam data, right?
# This collections list would work at NCSA, but I suspect it will need adjustment
# elsewhere. Needs to include both the raws and the `camera` dataset.
butler = Butler(
    "/repo/LSSTComCam/",
    collections=["LSSTComCam/raw/all"],
    writeable=True
)


In [3]:
intra_exposure_id = 2022030800001
extra_exposure_id = 2022030800002

In [4]:
# Replace "???" below by a query string that corresponds to all wavefront observations.
exposure_data_ids = set(
    butler.registry.queryDataIds(
        [
            "exposure"
        ], 
        where=f"exposure IN ({intra_exposure_id}, {extra_exposure_id})"
    )
)

In [5]:
exposure_data_ids

{{instrument: 'LSSTComCam', exposure: 2022030800001, ...},
 {instrument: 'LSSTComCam', exposure: 2022030800002, ...}}

In [6]:
instrument = Instrument.fromName("LSSTComCam", registry=butler.registry)

In [7]:
config = DefineVisitsTask.ConfigClass()

In [8]:
instrument.applyConfigOverrides(DefineVisitsTask._DefaultName, config)

In [9]:
config.groupExposures.name = "one-to-one"

In [10]:
task = DefineVisitsTask(config=config, butler=butler)

In [11]:
task.run(exposure_data_ids)

In [ ]:
data_id = {'instrument': 'LSSTComCam', 'detector': 0, 'exposure': 2022012800009}

In [ ]:
metadata = butler.get('raw.visitInfo', dataId=data_id, collections=["LSSTComCam/raw/all"])
print(metadata)